# Análisis no supervisado de Licitaciones

Se buscará encontrar comportamientos dentro de las licitaciones para identificar anomalías.

## Variables 

- Comparación de fechas
- Montos (normalizados por licitación/categoría)
- Frecuencia de ganador
- Empresas ganadoras en más de un rubro
- Recurrencia de convenios modificatorios
- Monto de convenios modificatorios

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
lic = pd.read_csv('data/raw/Copia de AGOSTO COMPRAS MUNICIPALES 2016-2019-SOCIAL TIC.xlsx - Compras.csv')

### Extracción de datos

In [100]:
datos = (
    pd.DataFrame()
    .assign(
        dep = lic['DEPENDENCIA SOLICITANTES'],
        proveedor = lic['PROVEEDOR'],
        cat = lic['CATEGORÍA '],
        sub_cat = lic['SUBCATEGORÍA'],
        comienzo_admon = lic['AÑO DE INICIO DE LA ADMINISTRACIÓN'],
        fin_admon = lic['AÑO DE TÉRMINO DE LA ADMINISTRACIÓN'],
        ejercicio = lic['EJERCICIO'],
        inicio_contrato = pd.to_datetime(lic['PERIODO DE CONTRATO: INICIO'],format='%m/%d/%Y', errors='coerce'),
        fin_contrato = pd.to_datetime(lic['PERIODO DE CONTRATO: TERMINACIÓN'],format='%m/%d/%Y', errors='coerce'),
        fecha_contrato = pd.to_datetime(lic['FECHA DE CONTRATO'],format='%m/%d/%Y', errors='coerce'),
        fecha_lic = pd.to_datetime(lic['ACTA ORIGINAL / FECHA'],format='%m/%d/%Y', errors='coerce'),
        monto_original = lic['MONTO ORIGINAL']
            .str.extractall('(\d*)')
            .fillna('').reset_index()
            .groupby(['level_0'])[0].apply(lambda x: ''.join(x)).astype(int)/100,
        monto_adicional = lic['CONVENIO MODIFICATORIO / MONTO ADICIONAL']
            .str.extractall('(\d*)')
            .fillna('').reset_index()
            .groupby(['level_0'])[0].apply(lambda x: ''.join(x))
            .astype(int)/100,
    )
)
datos.sample(5)

,dep,proveedor,cat,sub_cat,comienzo_admon,fin_admon,ejercicio,inicio_contrato,fin_contrato,fecha_contrato,fecha_lic,monto_original,monto_adicional
538,DIRECCIÓN GENERAL DE OBRAS PÚBLICAS,"MB CONSTRUYE VERDE, S.A. DE C.V.",OBRAS COMPLEMENTARIAS,CONSTRUCCIÓN (CANCHA Y MAMPOSTERÍA),2016.0,2018.0,2018,2018-06-01,2018-06-30,2018-05-18,2018-05-24,870909.06,10608.23
308,DIRECCIÓN DE MANTENIMIENTO MECÁNICO,"COMPAÑÍA CO, S.A. DE C.V.",MANTENIMIENTO VEHICULAR,MANTENIMIENTO VEHICULAR,2016.0,2018.0,2017,2017-10-24,2017-11-08,2017-10-24,NaT,725000.00,NaN
445,TESORERÍA MUNICIPAL,"OPERADORA COMERCIALIZADORA TEPEPAN, S.A. DE C.V.",SERVICIOS PROFESIONALES,SERVICIOS PROFESIONALES (SISTEMA SOFTWARE),2018.0,2021.0,2019,2019-02-15,2019-12-31,2019-02-15,NaT,9000000.00,NaN
369,DIRECCIÓN GENERAL DE OBRAS PÚBLICAS,"CONSTRUCTORA FIGOSA, S.A. DE C.V.",PAVIMENTACIÓN,PAVIMENTACIÓN,2018.0,2021.0,2018,NaT,NaT,2018-12-13,NaT,4566062.28,NaN
331,DIRECCIÓN DE RECURSOS HUMANOS,C. ENRIQUE EDUARDO SIEBERT SEPÚLVEDA,SERVICIOS PROFESIONALES,SERVICIOS PROFESIONALES (CAPACITACIÓN),2018.0,2021.0,2018,2018-10-31,2018-11-30,2018-10-31,2018-10-31,224112.00,NaN


### Creación de variables

In [148]:
# tfidf proveedores vs categorías

docs = (
    datos
    .assign(prov=lambda x: x.proveedor.str.replace(' ', '_').str.replace(',', '_').str.replace('.', '_').astype(str))
    .groupby('cat')
    .prov
    .apply(lambda x: ' '.join(x))
)

tfidf = TfidfVectorizer()
tfidf.fit(docs)

X = (
    pd.DataFrame.from_dict(
        dict(tfidf.transform(docs).todok()),
             orient='index').reset_index().assign(
            row=lambda x: x['index'].str[0],
            col=lambda x: x['index'].str[1]).assign(
            prov=lambda x: x.col.apply(lambda y: list(tfidf.get_feature_names())[y]),
            cat=lambda x: x.row.apply(lambda y: list(docs.index)[y])
        ).rename(columns={0: 'tfidf'})
    [['prov','tfidf','cat']]
    .merge(
        datos
        .assign(prov=lambda x: x.proveedor.str.replace(' ', '_').str.replace(',', '_').str.replace('.', '_').str.lower().astype(str)))
    .drop(columns=['prov'])
)
X.head()

,tfidf,cat,dep,proveedor,sub_cat,comienzo_admon,fin_admon,ejercicio,inicio_contrato,fin_contrato,fecha_contrato,fecha_lic,monto_original,monto_adicional
0,0.157412,PUBLICIDAD EN MEDIOS,COORDINACIÓN GENERAL DE COMUNICACIÓN SOCIAL,"26 DE MÉXICO, S. DE R.L. DE C.V.",PUBLICIDAD TELEVISIÓN,2018.0,2021.0,2019,2019-02-11,2019-12-31,2019-02-11,2019-02-11,1000000.0,NaN
1,0.157412,PUBLICIDAD EN MEDIOS,COORDINACIÓN GENERAL DE COMUNICACIÓN SOCIAL,"26 DE MÉXICO, S. DE R.L. DE C.V.",PUBLICIDAD TELEVISIÓN,2016.0,2018.0,2018,2018-01-01,2018-08-31,2018-02-02,2018-02-02,720000.0,NaN
2,0.157412,PUBLICIDAD EN MEDIOS,COORDINACIÓN GENERAL DE COMUNICACIÓN SOCIAL,"26 DE MÉXICO, S. DE R.L. DE C.V.",PUBLICIDAD RADIO / TELEVISIÓN,2016.0,2018.0,2017,2017-01-01,2017-12-31,2017-04-07,2017-04-07,790000.0,NaN
3,0.108170,VEHÍCULOS,SECRETARÍA DE SEGURIDAD PÚBLICA,"399 PROJEY DEVELOPMENT, S.A. DE C.V.",VEHÍCULOS,2018.0,2021.0,2018,2018-12-20,2018-12-31,2018-12-20,NaT,3572800.0,NaN
4,0.342677,COMUNICACIÓN,COORDINACIÓN GENERAL DE COMUNICACIÓN SOCIAL,"AB ESTUDIO DE COMUNICACIÓN, S.A. DE C.V.",ENCUESTA Y ESTUDIO DE OPINIÓN,2016.0,2018.0,2018,2018-01-01,2018-08-31,2018-01-12,2018-01-12,2000000.0,NaN


In [161]:
(X.fin_admon+1).astype(str).str.replace('\.0', '')+"0101"

0      20220101
1      20190101
2      20190101
3      20220101
4      20190101
5      20220101
6      20190101
7      20190101
8      20190101
9      20190101
10     20190101
11     20190101
12     20190101
13     20190101
14     20190101
15     20190101
16     20190101
17     20190101
18     20190101
19     20190101
20      nan0101
21     20190101
22     20190101
23     20190101
24      nan0101
25      nan0101
26      nan0101
27     20190101
28     20190101
29     20190101
         ...   
876    20190101
877    20190101
878    20190101
879    20190101
880    20190101
881    20190101
882    20190101
883    20190101
884    20190101
885    20190101
886    20190101
887    20190101
888    20190101
889    20190101
890     nan0101
891    20190101
892    20190101
893    20190101
894    20190101
895    20190101
896    20190101
897    20190101
898    20190101
899    20220101
900     nan0101
901     nan0101
902     nan0101
903    20190101
904    20190101
905    20190101
Name: fin_admon, Length:

In [167]:
(
    X
    .assign(duracion_contrato = lambda x: (x.fin_contrato - x.inicio_contrato).dt.days)
    .assign(porc_dentro_admon = lambda x: 
            ((pd.to_datetime((x.fin_admon+1).astype(str).str.replace('\.0', '')+"0101",errors='coerce') - x.inicio_contrato).dt.days / x.duracion_contrato)
           .apply(lambda y: min(y,1)))
).sort_values('porc_dentro_admon')

,tfidf,cat,dep,proveedor,sub_cat,comienzo_admon,fin_admon,ejercicio,inicio_contrato,fin_contrato,fecha_contrato,fecha_lic,monto_original,monto_adicional,duracion_contrato,porc_dentro_admon
128,0.327287,VIALIDADES: PAVIMENTACIÓN / BACHEO / BANQUETAS,DIRECCIÓN GENERAL DE OBRAS PÚBLICAS,C. FRANCISCO JAVIER ACOSTA REYES,MEZCLA ASFÁLTICA,2016.0,2018.0,2017,2017-12-31,2017-04-30,2017-01-06,NaT,3.122103e+06,NaN,-245.0,-1.493878
0,0.157412,PUBLICIDAD EN MEDIOS,COORDINACIÓN GENERAL DE COMUNICACIÓN SOCIAL,"26 DE MÉXICO, S. DE R.L. DE C.V.",PUBLICIDAD TELEVISIÓN,2018.0,2021.0,2019,2019-02-11,2019-12-31,2019-02-11,2019-02-11,1.000000e+06,NaN,323.0,1.000000
605,0.128368,"CONSTRUCCIÓN, REHABILITACIÓN Y EQUIPAMIENTO A ...",DIRECCIÓN GENERAL DE OBRAS PÚBLICAS,"LAGUERA CONSTRUCCIONES, S.A. DE C.V.",CONSTRUCCIÓN DE DEPORTIVO,2016.0,2018.0,2017,2017-12-29,2018-03-28,2017-12-20,NaT,1.576323e+07,NaN,89.0,1.000000
606,0.238992,CUARTOS ROSAS / INDEPENDIENTES,DIRECCIÓN GENERAL DE OBRAS PÚBLICAS,"LAGUERA CONSTRUCCIONES, S.A. DE C.V.",CONSTRUCCIÓN DE CUARTOS,2016.0,2018.0,2018,2018-06-01,2018-08-29,2018-05-18,2018-05-17,1.098345e+07,NaN,89.0,1.000000
607,0.238992,CUARTOS ROSAS / INDEPENDIENTES,DIRECCIÓN GENERAL DE OBRAS PÚBLICAS,"LAGUERA CONSTRUCCIONES, S.A. DE C.V.",CONSTRUCCIÓN DE CUARTOS,2016.0,2018.0,2017,2017-10-10,2017-12-28,2017-09-28,2017-08-16,1.387379e+07,NaN,79.0,1.000000
608,0.087182,OBRAS COMPLEMENTARIAS,DIRECCIÓN GENERAL DE OBRAS PÚBLICAS,"LAGUERA CONSTRUCCIONES, S.A. DE C.V.",CONSTRUCCIÓN DE PUENTE PEATONAL,2016.0,2018.0,2017,2017-12-18,2018-02-15,2017-12-04,2017-12-01,2.317529e+06,NaN,59.0,1.000000
609,0.160172,PAVIMENTACIÓN,DIRECCIÓN GENERAL DE OBRAS PÚBLICAS,"LAGUERA CONSTRUCCIONES, S.A. DE C.V.",Pavimentación,2016.0,2018.0,2017,2017-07-24,2017-12-20,2017-07-21,2017-07-21,2.603048e+06,NaN,149.0,1.000000
610,0.160172,PAVIMENTACIÓN,DIRECCIÓN GENERAL DE OBRAS PÚBLICAS,"LAGUERA CONSTRUCCIONES, S.A. DE C.V.",Pavimentación,2016.0,2018.0,2017,2017-07-24,2017-12-20,2017-07-21,2017-07-21,1.235454e+06,287515.09,149.0,1.000000
611,0.160172,PAVIMENTACIÓN,DIRECCIÓN GENERAL DE OBRAS PÚBLICAS,"LAGUERA CONSTRUCCIONES, S.A. DE C.V.",Pavimentación,2016.0,2018.0,2017,2017-10-10,2017-12-28,2017-09-27,2017-08-16,1.856021e+06,NaN,79.0,1.000000
614,0.114226,COMUNICACIÓN,COORDINACIÓN GENERAL DE COMUNICACIÓN SOCIAL,"LITOGRÁFICA VOZ, S.A. DE C.V.",SERVICIOS DE IMPRENTA,2016.0,2018.0,2017,2017-10-01,2017-12-31,2017-10-23,NaT,1.800000e+06,NaN,91.0,1.000000


In [70]:
lic.sample(2).T

,450,530
FUENTE DE INFORMACION,84312018,98462017
ADMINISTRACIÓN,2016-2018,2016-2018
AÑO DE INICIO DE LA ADMINISTRACIÓN,2016,2016
AÑO DE TÉRMINO DE LA ADMINISTRACIÓN,2018,2018
UNIDAD ADMINISTRATIVA CONVOCANTE,OFICIALÍA MAYOR,DIRECCIÓN GENERAL DE OBRAS PÚBLICAS
DEPENDENCIA SOLICITANTES,TESORERÍA MUNICIPAL,DIRECCIÓN GENERAL DE OBRAS PÚBLICAS
DEPENDENCIA GENERAL SOLICITANTES,TESORERÍA MUNICIPAL,DIRECCIÓN GENERAL DE OBRAS PÚBLICAS
TITULAR DE LA DEPENDENCIA SOLICITANTE,NaN,NaN
ACTA ORIGINAL / FECHA,7/3/2018,7/17/2017
ACTA ORIGINAL / DOCUMENTO,NaN,http://www.planjuarez.org/adquisiciones/LO-808...
